In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import graph as gr
import utils as ut
import feature_selection as fs
import classification as cl
import os.path
import matplotlib.pyplot as plt
from pprint import pprint
import importlib
import community
import seaborn as sns
from gensim.models import KeyedVectors
from pyemd import emd
from gensim import corpora
from nltk.corpus import stopwords
from gensim.matutils import softcossim
import copy
import itertools
import pickle
import PathSim as ps
import importlib

Using Theano backend.


In [2]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Reshape
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import load_model

In [7]:
#########dataset and domain selection : sleep apnea: 0 ; obesity:1
importlib.reload(ut)
select_domain  = 0   ###[set 0 for apnea, 1 for obesity]
domain = ['apnea','obesity']
root = '../linkPrediction/'
filepath=[['dataset/apnea-all,3.csv',
         'dataset/apnea-distinct_keyword.csv'],
        ['dataset/obesity-all,3.csv',
         'dataset/obesity-distinct_keyword.csv']]
########path to save graph files
graphpath=root+'graphs/'+domain[select_domain]
########path to save result files, data files and resultent figures
datapath=root+'dataframes/'+domain[select_domain]
#path to save classification models
modelpath=root+'models/'+domain[select_domain]
########columns with sub-columns
column_split=['keyword','author_name','affiliation_1','affiliation_2','country']
#time=[parent_year, train:start_year, test:start_year, no_of_test_years(1), total_year_in_each_iteration, no of grandparents]
time=[2007,2008,2015,1,1,20]
times=time
######load dataset and keyword list into dataframes
df, key_list = ut.load_dataset(filepath[select_domain], column_split)

In [ ]:
#######graph building and saving in graphpath: train graph, test graph and parent graph(previous year of train graph)  
importlib.reload(gr)
time=[2007,2008,2015,1,1,20]
gr.dynamic_train_test_graph_build(df, key_list, graphpath, time)

In [8]:
# time=[1991,1992,1994,1,1,20]
importlib.reload(gr)
g_train, g_test, g_parent, g_train_static, g_test_static = gr.graph_load(graphpath, time)
# g_train, g_test = gr.graph_norm(root, domain, select_domain, g_train, g_test)

../linkPrediction/graphs/apnea\train_graph_2008.gpickle loaded, nodes: 459 edges: 773
../linkPrediction/graphs/apnea\train_graph_2009.gpickle loaded, nodes: 470 edges: 688
../linkPrediction/graphs/apnea\train_graph_2010.gpickle loaded, nodes: 496 edges: 786
../linkPrediction/graphs/apnea\train_graph_2011.gpickle loaded, nodes: 536 edges: 977
../linkPrediction/graphs/apnea\train_graph_2012.gpickle loaded, nodes: 584 edges: 1054
../linkPrediction/graphs/apnea\train_graph_2013.gpickle loaded, nodes: 634 edges: 1280
../linkPrediction/graphs/apnea\train_graph_2014.gpickle loaded, nodes: 583 edges: 1144
../linkPrediction/graphs/apnea\test_graph_2015.gpickle loaded, nodes: 1093 edges: 2019
../linkPrediction/graphs/apnea\parent_graph_2007-2008.gpickle loaded, nodes: 388 edges: 689
../linkPrediction/graphs/apnea\train_graph_2008-2015.gpickle loaded, nodes: 1093 edges: 5142
../linkPrediction/graphs/apnea\test_graph_2015-2016.gpickle loaded, nodes: 1093 edges: 2019


In [5]:
s_key={0:[9875,1020],1:[7614,9953]}
for t in range(2008,2015):
    g_train[t].remove_nodes_from(s_key[select_domain])
g_test[2015].remove_nodes_from(s_key[select_domain])
g_parent.remove_nodes_from(s_key[select_domain])
g_train_static.remove_nodes_from(s_key[select_domain])
g_test_static.remove_nodes_from(s_key[select_domain])

In [ ]:
importlib.reload(fs)
importlib.reload(cl)
#ration of negative:positive train instance:(eg,10 for 10:1=neg:pos)
freq=10
##### time=[2001,2007,2015,1,1,10]
##### train_data building: dictionary of each years non-connected node pairs dataframes
train_data, train_data_static, parent_data, test_data_static, edge_list = cl.non_edge_feature_dataframe(g_train, 
                                                                                                         g_test,
                                                                                                         g_parent,
                                                                                                         g_train_static, 
                                                                                                         g_test_static,  
                                                                                                         time, 
                                                                                                         freq)
#### train_data: non-connected node pairs with feature values
####node_features: node feature values
node_feature, train_data = fs.dynamic_graph_feature_set(df, key_list, train_data, g_parent, g_train, g_train_static, time)

### save train data (edge data and node data) and edge list

importlib.reload(ut)
ut.save_data(train_data, datapath, domain[select_domain], "train_data", time)
ut.save_data(node_feature, datapath, domain[select_domain], "node_feature", time)
ut.save_data(edge_list, datapath, domain[select_domain], "edge_list", time)

### test data generation

In [6]:
importlib.reload(fs)
importlib.reload(cl)

edge_list = ut.load_data(datapath, domain[select_domain], "edge_list", time)
edge_set = set(edge_list)
edges = list(g_test[2015].edges())
non_edges =  list(nx.non_edges(g_test[2015]))
r_edges = [(b, a) for a, b in edges]
r_non_edges = [(b, a) for a, b in non_edges]
test_edges = []
label = []
train_ed=[]
ins = set(edges).intersection(edge_set)
train_ed.extend(ins)
label.extend([1]*len(ins))
ins = set(r_edges).intersection(edge_set)
train_ed.extend(ins)
label.extend([1]*len(ins))
ins = set(non_edges).intersection(edge_set)
train_ed.extend(ins)
label.extend([0]*len(ins))
ins = set(r_non_edges).intersection(edge_set)
train_ed.extend(ins)
label.extend([0]*len(ins))
train_data={}
train_data[2015] = pd.DataFrame({'row_name': train_ed, 'label': label})

g_train[2015]=g_test[2015]
time=[2014,2015,2016,1,1,20]
test_node, test_data = fs.dynamic_graph_feature_set(df, key_list, train_data, g_train[2014], g_train, g_train_static, time)

../linkPrediction/dataframes/apnea


### Save test data (edge data and node data)

In [ ]:
importlib.reload(ut)
time=[2007,2008,2015,1,1,20]
ut.save_data(test_data, datapath, domain[select_domain], "test_data", time)
ut.save_data(test_node, datapath, domain[select_domain], "test_node", time)

### Load train, test and edge data from saved files

In [9]:
importlib.reload(ut)
train_data = ut.load_data(datapath, domain[select_domain], "train_data", times)
node_feature = ut.load_data(datapath, domain[select_domain], "node_feature", times)
edge_list = ut.load_data(datapath, domain[select_domain], "edge_list", times)
test_data = ut.load_data(datapath, domain[select_domain], "test_data", times)
test_node = ut.load_data(datapath, domain[select_domain], "test_node", times)
print('train data length:',len(train_data))
print('node feature length:',len(node_feature))
print('edge_list length:',len(edge_list))
print('test_data length:',len(test_data[2015]))
print('test_node length:',len(test_node[2015]))

../linkPrediction/dataframes/apnea
../linkPrediction/dataframes/apnea
../linkPrediction/dataframes/apnea
../linkPrediction/dataframes/apnea
../linkPrediction/dataframes/apnea
train data length: 7
node feature length: 7
edge_list length: 12494
test_data length: 12494
test_node length: 1093


### feature data reshape function to transform dataframes for LSTM training

In [10]:
def reshape(train_data, node_feature, edge_list, test_data,test_node,times,features):
    cat ={25:0,5:1,3:2,1:3} ####predefined geneoligical heredity score (eg. 25 for grandparent, 5 for parent)
    ts = times[1]
    te = times[2]+1
    it_index = times[4]
    times_range = te - ts
    total_sample = len(edge_list)
    train_data[times[2]] = test_data
    node_feature[times[2]] = test_node
    feature_length = len(features['edge'])+32
    X = np.zeros([total_sample, times_range, feature_length])
    y = np.zeros(total_sample)
    print("X shape:", X.shape, "y shape:", y.shape)
    for id, edge in enumerate(edge_list):
        for t in range(ts, te, it_index):
            t_data = train_data[t]
            n_data = node_feature[t]
            node_list = set(n_data['node_index'])
            edge_list_t = set(t_data['row_name'])
            if edge in edge_list_t:
                train_row = np.asarray(t_data.loc[t_data['row_name'] == edge,features['edge']].values[0])
                X[id][t - ts][0:6] = train_row
                X[id][t - ts][3] = X[id][t - ts][3]*(t-ts+1)*2
            if edge[0] in node_list:
                node_row0 = np.asarray(n_data.loc[n_data['node_index'] == edge[0],features['node']].values[0])
                X[id][t - ts][6:9] = node_row0[0:3]
                X[id][t - ts][cat[node_row0[3]]+9] = 1
                X[id][t - ts][cat[node_row0[4]]+13] = 1
                X[id][t - ts][cat[node_row0[5]]+17] = 1
                X[id][t - ts][21] = node_row0[6]
                if edge[1] not in node_list:
                    X[id][t - ts][3] = node_row0[6]*(t-ts+1)
            if edge[1] in node_list:
                node_row1 = np.asarray(n_data.loc[n_data['node_index'] == edge[1],features['node']].values[0])
                X[id][t - ts][22:25] = node_row1[0:3]
                X[id][t - ts][cat[node_row1[3]]+25] = 1
                X[id][t - ts][cat[node_row1[4]]+29] = 1
                X[id][t - ts][cat[node_row1[5]]+33] = 1
                X[id][t - ts][37] = node_row1[6]
                if edge[0] not in node_list:
                    X[id][t - ts][3] = node_row1[6]*(t-ts+1)
        y[id] = test_data.loc[test_data['row_name'] == edge,'label'].values[0]
    return X, y

### node and edge feature catagories 

In [11]:
features = [{'node':['term_aut','node_type_aut'],
           'edge':['typeaut']},
           {'node':['term_art','node_type_art'],
           'edge':['typeart']},
           {'node':['degree','node_type_deg'],
           'edge':['typenode']},
           {'node':['citation'],
           'edge':['citation1']},
           {'node':['degree'],
           'edge':['pref']},
           {'node':['degree'],
           'edge':['cm']}]

### Reshaping dataframe and putting features into the specific column index location

In [ ]:
#####################  6           7        8       9,10,11,12     13,14,15,16      17,18,19,20      21
#####################  22          23       24      25,26,27,28    29,30,31,32      33,34,35,36      37
feature = {'node':['term_aut','term_art','degree','node_type_aut','node_type_art','node_type_deg','citation'],
           'edge':['typeaut','typeart','typenode','citation1','pref','cm']}
################       0         1         2           3        4      5

start = tm.time()
X, y = reshape(train_data, node_feature, edge_list, test_data[2015],test_node[2015],times,feature)
end = tm.time()
print(end - start)

### saving reshaped data

In [ ]:
ut.save_data(X, datapath, domain[select_domain], "X-features", times)
ut.save_data(y, datapath, domain[select_domain], "y-features", times)

### Column index for each feature type in reshaped X feature array

In [13]:
feature_names = {
'author':[0,6,22,9,10,11,12,25,26,27,28],
'article': [1,7,23,13,14,15,16,29,30,31,32],
'degree': [2,8,24,17,18,19,20,33,34,35,36],
'citation': [3,21,37],
'pref': [4,8,24],
'cm': [5,8,24]
}

### functions for LSTM training

In [14]:
import keras

class CustomSaver(keras.callbacks.Callback):
    
    def __init__(self, param):
        self.name = param[3]
        self.times = param[4]
    def on_epoch_end(self, epoch, logs={}):
        ep_set =set([100,500,1000])
        if epoch in ep_set:  # or save after some epoch, each k-th epoch etc.
            ut.save_data(self.model, datapath, domain[select_domain], "model-"+name+"-"+str(epoch), self.times)

def create_linear(param):
        inputx = Input(shape=(param[0], param[1]))
        x = LSTM(20)(inputx)
        x = LSTM(20, activation="relu")(inputx)
        x = Dense(40, activation="relu")(x)
        x = Dense(20, activation="relu")(x)
        x = Dense(param[2],activation='linear',name="lin")(x)
        model = Model(inputs=inputx, outputs=x)
        return model

def create_cat1(param):
        inputx = Input(shape=(param[0], param[1]))
        x = LSTM(20)(inputx)
        x = Dense(20)(x)
        x = Dense(10,activation='relu')(x)
        x = Dense(param[2],activation='softmax',name="cat1")(x)
        model = Model(inputs=inputx, outputs=x)
        return model
    
def create_cat2(param):
        inputx = Input(shape=(param[0], param[1]))
        x = LSTM(20)(inputx)
        x = Dense(20)(x)
        x = Dense(10,activation='relu')(x)
        x = Dense(param[2],activation='softmax',name="cat2")(x)
        model = Model(inputs=inputx, outputs=x)
        return model
    
def lstm_forecast(X,param):
        X = ut.scale(X,0,1)
        y = X[:,7]
        X = X[:,0:7]
        batch_size = param[1]
        epoch = param[2]
        names1 = set(['author','article','degree'])
        names2 = set(['citation','pref','cm'])
        print(X.shape,y.shape)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = param[0], random_state = 0)
        ######## the first branch operates on the linear input

        lin_parameters = [X_train[:,:,0:3].shape[1], X_train[:,:,0:3].shape[2], y_train[:,0:3].shape[1]]
        lin = create_linear(lin_parameters)        
        
        saver = CustomSaver(param)
        
        if param[3] in names1:
            cat_parameters1 = [X_train[:,:,3:7].shape[1], X_train[:,:,3:7].shape[2], y_train[:,3:7].shape[1]]
            cat1 = create_cat1(cat_parameters1)

            cat_parameters2 = [X_train[:,:,7:11].shape[1], X_train[:,:,7:11].shape[2], y_train[:,7:11].shape[1]]
            cat2 = create_cat2(cat_parameters2)
        
            model = Model(inputs=[lin.input, cat1.input, cat2.input], outputs=[lin.output, cat1.output,cat2.output])
            model.compile(loss={'lin':'mse','cat1':'categorical_crossentropy','cat2':'categorical_crossentropy'},
                              optimizer='Adam',
                              metrics={'lin':'accuracy','cat1':'categorical_accuracy','cat2':'categorical_accuracy'})
            history = model.fit([X_train[:,:,0:3],X_train[:,:,3:7],X_train[:,:,7:11]],
                                [y_train[:,0:3],y_train[:,3:7],y_train[:,7:11]],
                                callbacks=[saver],
                                batch_size=batch_size,
                                epochs=epoch,
                                verbose=1)
        else:
            model = Model(inputs=lin.input, outputs=lin.output)
            model.compile(loss={'lin':'mse'},
                              optimizer='Adam',
                              metrics={'lin':'accuracy'})
            history = model.fit(X_train[:,:,0:3],
                                y_train[:,0:3],
                                callbacks=[saver],
                                batch_size=batch_size,
                                epochs=epoch,
                                verbose=1)
#         y_pr = model.predict([X_test[:,:,0:3],X_test[:,:,3:7],X_test[:,:,7:11]])
        return X_test, y_test

### Timeseries forecasting Training and saving to files

In [ ]:
feature_names = {
# 'author':[0,6,22,9,10,11,12,25,26,27,28]
'article': [1,7,23,13,14,15,16,29,30,31,32]
# 'degree': [2,8,24,17,18,19,20,33,34,35,36],
# 'citation': [3,21,37],
# 'pref': [4,8,24],
# 'cm': [5,8,24]
}
names1 = set(['author','article','degree'])
names2 = set(['citation','pref','cm'])
X = ut.load_data(datapath, domain[select_domain], "X-features", times)
y = ut.load_data(datapath, domain[select_domain], "y-features", times)
for name,feature in feature_names.items():
    ### running for only first 3 features
    if name in names1:
        param = [0.3,64,1001,name,times]
        print(name,"---------------------------------------------------------------------------------")
        X_test, y_test = lstm_forecast(X[:,:,feature],param)
        ut.save_data(X_test, datapath, domain[select_domain], "X_test-"+name, times)
        ut.save_data(y_test, datapath, domain[select_domain], "y_test-"+name, times)

### Printing RMSE values for the forecasted results

In [ ]:
#obesity =1, apnea =0 [select_domain=0 or 1]
feature_names = {
'author':[0,6,22,9,10,11,12,25,26,27,28],
'article': [1,7,23,13,14,15,16,29,30,31,32],
'degree': [2,8,24,17,18,19,20,33,34,35,36],
'citation': [3,21,37],
'pref': [4,8,24],
'cm': [5,8,24]
}
names1 = set(['author','article','degree'])
names2 = set(['citation','pref','cm'])
epochs = [100,500,1000]
for name,feature in feature_names.items():
    X_test = ut.load_data(datapath, domain[select_domain], "X_test-"+name, times)
    y_test = ut.load_data(datapath, domain[select_domain], "y_test-"+name, times)
    for ep in epochs:      
        if name in names1:
            m=ut.load_data(datapath, domain[select_domain], "model-"+name+"-"+str(ep), times)    
            y_pr = m.predict([X_test[:,:,0:3],X_test[:,:,3:7],X_test[:,:,7:11]])
            mse = mean_squared_error(y_test[:,0],y_pr[0][:,0])
            rmse1 = sqrt(mse)
            mse = mean_squared_error(y_test[:,1],y_pr[0][:,1])
            rmse2 = sqrt(mse)
            print(name,"-",ep,":",rmse1," --- ",rmse2)
        if name in names2:
            m=ut.load_data(datapath, domain[select_domain], "model-"+name+"-"+str(ep), times)
            y_pr = m.predict(X_test)
            mse = mean_squared_error(y_test[:,0],y_pr[:,0])
            rmse1 = sqrt(mse)
            mse = mean_squared_error(y_test[:,1],y_pr[:,1])
            rmse2 = sqrt(mse)
            print(name,"-",ep,":",rmse1," --- ",rmse2)

### function for link classification using the forecasted result

In [15]:
def create_class(param):
    inputx = Input(shape=(param[1],))
    x = Dense(20, activation='relu')(inputx)
    x = Dense(10,activation='relu')(x)
    x = Dense(1,activation='sigmoid',name="class")(x)
    model = Model(inputs=inputx, outputs=x)
    return model

def lstm_classification(X,y,param):
    batch_size = param[1]
    epoch = param[2]
    print(X.shape,y.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = param[0], random_state = 0)
    class_param = [X.shape[0], X.shape[1], 1]
    print(class_param)
    
    cl = create_class(class_param)
    model = Model(inputs=cl.input, outputs=cl.output)
    model.compile(loss={'class':'binary_crossentropy'},
                      optimizer='Adam',
                      metrics={'class':'accuracy'})
    print(X_train.shape,y_train.shape)
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        shuffle=True,
                        batch_size=batch_size,
                        epochs=epoch,
                        verbose=1)
    return X_test, y_test, model
        

### Link classification training and saving into files

In [ ]:
feature_names = {
'author':[0,6,22,9,10,11,12,25,26,27,28],
'article': [1,7,23,13,14,15,16,29,30,31,32],
'degree': [2,8,24,17,18,19,20,33,34,35,36],
'citation': [3,21,37],
'pref': [4,8,24],
'cm': [5,8,24]
}

results = []
names1 = set(['author','article','degree'])
names2 = set(['citation','pref','cm'])
X = ut.load_data(datapath, domain[select_domain], "X-features", times)
y = ut.load_data(datapath, domain[select_domain], "y-features", times)
for name,feature in feature_names.items():
    param = [0.3,64,500,name,times]
    m=ut.load_data(datapath, domain[select_domain], "model-"+name+"-"+str(1000), times)
    X1=X[:,:,feature]
    X1=X1[:,0:7]
    print(name, X1.shape, "--------------------------------------------------------------------")
    if name in names1:
        y_pr = m.predict([X1[:,:,0:3],X1[:,:,3:7],X1[:,:,7:11]])
#         row_maxes = y_pr[1].max(axis=1).reshape(-1, 1)
#         y_pr[1][:] = np.where(y_pr[1] == row_maxes, 1, 0)
#         row_maxes = y_pr[2].max(axis=1).reshape(-1, 1)
#         y_pr[2][:] = np.where(y_pr[2] == row_maxes, 1, 0)
        y_pr1 = np.concatenate((y_pr[0],y_pr[1],y_pr[2]),axis=1)
    if name in names2:
        y_pr1 = m.predict(X1)
    X_test, y_test, model = lstm_classification(y_pr1,y,param)
    results.append(cl.model_evaluate(model, X_test, y_test, param[1], name))
ut.save_data(results, datapath, domain[select_domain], "results", times)

### Loading classification result files and printing auc and accuracy score

In [ ]:
results = ut.load_data(datapath, domain[select_domain], "results", times)
for result in results:
    print(result['model name'], result['auc'], result['test accuracy'])